<a href="https://www.kaggle.com/code/mohdmuttalib/novozymes-enzyme-stability-prediction?scriptVersionId=142735869" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Contact Information

- **Name** - Mohd Muttalib
- **Phone** - +91-8445818187
- **Email** - muttalib1326@gmail.com
- **Portfolio** -[link](https://www.kaggle.com/mohdmuttalib)

<center><img src="https://storage.googleapis.com/kaggle-competitions/kaggle/37190/logos/header.png?t=2022-08-30-15-34-26" width=1000></center>



<div class="alert alert-block alert-info">
<center><h1>Novozymes Enzyme Stability Prediction</h1></center>
<center><h3>Help identify the thermostable mutations in enzymes</h3></center>
</div>
<div class="alert alert-block alert-success">
<center><h1>EDA + XGBoost</h1></center>
</div>


### <h1>Table of contents</h1><a class='anchor' id='top'></a>
- [Import libraries](#import)
- [Utils](#utils)
- [Config](#config)
- [Load data](#load)
- [Train profiling](#train_profiling)
- [Test profiling](#test_profiling)
- [What is PDB?](#pdb)
- [Label Encode](#labelencode)
- [XGBoost Regressor](#xgboost)
- [Evaluate: Spearman's rank correlation coefficient](#evaluate)
- [Predict](#predict)
- [Submission](#submission)


<div class="alert alert-block alert-warning">  
<b>Note:</b> This is a work in progress ⚠️
</div>

<div class="alert alert-block alert-info">
<center><h1>What is the goal of this competition?</h1></center>
</div>

<div class="alert alert-block alert-info">
The goal of this competition is to predict the thermal stability of enzyme variants. The experimentally measured thermal stability (melting temperature) data includes natural sequences, as well as engineered sequences with single or multiple mutations upon the natural sequences.

A point mutation is a genetic mutation where a single nucleotide base is changed, inserted or deleted from a DNA or RNA sequence of an organism's genome. Point mutations have a variety of effects on the downstream protein product—consequences that are moderately predictable based upon the specifics of the mutation.

These consequences can range from no effect (e.g. synonymous mutations) to deleterious effects (e.g. frameshift mutations), with regard to protein production, composition, and function.
</div>




<div class="alert alert-block alert-success">
<center><h1>What is an enzyme?</h1></center>
</div>
<div class="alert alert-block alert-success">
Enzymes are proteins that act as biological catalysts by accelerating chemical reactions. An enzyme is therefore a molecule that increases the speed of biochemical reaction in an organism. The molecules upon which enzymes may act are called substrates, and the enzyme converts the substrates into different molecules known as products.
</div>
<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F3197853%2F011e8e6d88265aaf8d13c7b9cae4b2b8%2Fenzyme_substrate.jpeg?generation=1663958942493732&alt=media" width=800></center>
<div class="alert alert-block alert-success">
<center><h1>What is a protein?</h1></center>
</div>
<div class="alert alert-block alert-success">
Proteins are large biomolecules and macromolecules that comprise one or more long chains of amino acid residues. Proteins are also known as polypeptides.

Peptides are short chains of amino acids linked by peptide bonds. Chains of fewer than twenty amino acids are called oligopeptides, and include dipeptides, tripeptides, and tetrapeptides. So, a polypeptide is a longer, continuous, unbranched peptide chain.
</div>
<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F3197853%2F649adf375f36aaf3cad5064cf8d543eb%2Fprotein_structure.jpeg?generation=1663958961249770&alt=media" width=600></center>
<div class="alert alert-block alert-success">
<center><h1>What is an amino acid?</h1></center>
</div>
<div class="alert alert-block alert-success">
Amino acids are organic compounds that contain both amino and carboxylic acid functional groups. Although hundreds of amino acids exist in nature, by far the most prevalent are the alpha-amino acids, which comprise proteins.
</div>
<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F3197853%2Ff6f63060a29aa51e3675e36e27c51b9d%2Faminoacids.png?generation=1663958987152864&alt=media" width=600></center>



In [ ]:
!pip install biopandas

# Import libraries <a class='anchor' id='import'></a> [↑](#top)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_profiling
import time
import torch
import torch.nn as nn

# Utils <a class='anchor' id='utils'></a>[↑](#top)

In [ ]:
def sep():
    print("-"*100)

# Config <a class='anchor' id='config'></a>[↑](#top)

In [ ]:
class paths:
    TRAIN = "/kaggle/input/novozymes-enzyme-stability-prediction/train.csv"
    TEST = "/kaggle/input/novozymes-enzyme-stability-prediction/test.csv"
    SUBMISSION = "/kaggle/input/novozymes-enzyme-stability-prediction/sample_submission.csv"
    PDB_FILE = "/kaggle/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb"

# Load data <a class='anchor' id='load'></a>[↑](#top)

Let's load the competition's data.

In [ ]:
train_df = pd.read_csv(paths.TRAIN)
test_df = pd.read_csv(paths.TEST)
print(f"Train dataframe has shape: {train_df.shape}")
print(f"Test dataframe has shape: {test_df.shape}")
display(train_df.head())
display(test_df.head())

# Train profiling <a class='anchor' id='train_profiling'></a>[↑](#top)

- `protein_sequence`: has a high cardinality: 28981 distinct values ---> High cardinality
- `data_source`: has a high cardinality: 324 distinct values ---> High cardinality
- `data_source` has 3347 (10.7%) missing values ---> Missing
- `protein_sequence_len` is highly skewed (γ1 = 29.89928625) ---> Skewed
- `seq_id` is uniformly distributed ---> Uniform
- `protein_sequence` is uniformly distributed ---> Uniform
- `seq_id` has unique values

In [ ]:
train_df.profile_report()

# Test profiling <a class='anchor' id='test_profiling'></a>[↑](#top)

- `pH` has constant value "8" ---> Constant
- `data_source` has constant value "Novozymes" ---> Constant
- `protein_sequence` has a high cardinality: 2413 distinct values ---> High cardinality
- `pH` is highly correlated with data_source and 1 other fields ---> High correlation
- `data_source` is highly correlated with pH and 1 other fields ---> High correlation
- `protein_sequence_len` is highly correlated with pH and 1 other fields ---> High correlation
- `seq_id` is uniformly distributed ---> Uniform
- `protein_sequence` is uniformly distributed ---> Uniform
- `seq_id` has unique values ---> Unique
- `protein_sequence` has unique values

In [ ]:
test_df.profile_report()

### Plot target [↑](#top)

Let's plot our target so we can get a sense of its distribution. We can see that it's right skewed with a large value around 25.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = go.Figure()
colors = ["#00FFFB", "#002EFF"]

i = 0
# Add one subplot
fig.add_trace(
    go.Histogram(x=train_df["tm"],
               name="tm",
               hovertemplate = "target column" + ' %{y:.2f}',
               marker=dict(color=colors[i])
              )
)
fig.update_xaxes(
    title_text="Target column",
    title_font_color=colors[i],
    tickfont_color=colors[i]) 
fig.update_yaxes(
    title_text = "Frequency", 
    title_font_color=colors[i], 
    tickfont_color=colors[i]) 
    
fig.update_layout(height=800,
                  width=1000,
                  title_text="Target column: Higher tm means the protein variant is more stable.",
                  template="plotly_dark",
                  xaxis=dict(color="#FF9300"),
                  yaxis=dict(color="#FF9300"))

fig.update_traces(marker_line_width=0.1,marker_line_color=colors[1])
fig.show()

### Sequence length [↑](#top)

Sequence length seems to vary for the `train.csv` while it is constant for `test.csv`

We can see there is one heavy outlier of length `32767`. The distribution is highly right skewed.

In [ ]:
train_df["protein_sequence_len"] = train_df["protein_sequence"].apply(lambda x: len(x))
test_df["protein_sequence_len"] = test_df["protein_sequence"].apply(lambda x: len(x))
print("----------- TRAIN -----------")
display(train_df[["protein_sequence_len"]].describe())
print("----------- TEST -----------")
display(test_df[["protein_sequence_len"]].describe())

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


fig = go.Figure()
colors = ["#FF00F0", "#BD00FF"]

i = 0
# Add one subplot
fig.add_trace(
    go.Histogram(x=train_df["protein_sequence_len"],
               name="protein_sequence_len",
               hovertemplate = 'Protein Sequence Length:  %{x:.2f}' + '<br>Frequency: %{y:.2f}</br>',
               marker=dict(color=colors[i])
              )
)
fig.update_xaxes(
    title_text="Protein Sequence Length",
    title_font_color=colors[i],
    tickfont_color=colors[i],
    range=[0, 1500]
) 
fig.update_yaxes(
    title_text = "Frequency", 
    title_font_color=colors[i], 
    tickfont_color=colors[i]
) 

fig.update_layout(height=800,
                  width=1000,
                  title_text="Potein Sequence Length Histogram.",
                  template="plotly_dark",
                  xaxis=dict(color="#FF9300"),
                  yaxis=dict(color="#FF9300"))
fig.update_traces(marker_line_width=2,marker_line_color=colors[1])

fig.show()

# What is PDB? <a class='anchor' id='pdb'></a>[↑](#top)

PDB file format

In the PDB data file format for macromolecular models, each atom is designated either ATOM or HETATM (which stands for hetero atom).

ATOM is reserved for atoms in standard residues of protein, DNA or RNA.

HETATM is applied to non-standard residues of protein, DNA or RNA, as well as atoms in other kinds of groups, such as carbohydrates, substrates, ligands, solvent, and metal ions.

In [ ]:
from biopandas.pdb import PandasPdb

pdb_df =  PandasPdb().read_pdb(paths.PDB_FILE)
pdb_df.df.keys()

### Get dataframes

The `pdb` file contains data which we can convert into new dataframes. Let's check it out.

In [ ]:
atom_df = pdb_df.df['ATOM']
hetatm_df = pdb_df.df['HETATM']
anisou_df = pdb_df.df['ANISOU']
others_df = pdb_df.df['OTHERS']
print(f"ATOM dataset is of shape: {atom_df.shape}"), sep()
print(f"HETATM dataset is of shape: {hetatm_df.shape}"), sep()
print(f"ANISOU dataset is of shape: {anisou_df.shape}"), sep()
print(f"OTHERS dataset is of shape: {others_df.shape}"), sep()
display(atom_df.head())
display(hetatm_df.head())
display(anisou_df.head())
display(others_df.head())

### Plot sequence

In [ ]:
import plotly.express as px

fig = px.scatter_3d(atom_df, x = "x_coord",
                    y = "y_coord",
                    z = "z_coord",
                    color = "element_symbol",
                    color_discrete_sequence = ["#84FFA9", "#00FFF7", "#003AFF", "#F000FF", "#FBFF00"])
fig.update_traces(marker = dict(size = 3))
fig.update_coloraxes(showscale = False)
fig.update_layout(template = "plotly_dark")
fig.show()

### Get sequences [↑](#top)

We will only use sequences lower than 221.

In [ ]:
from scipy.sparse import csr_matrix

train_df = train_df[train_df["protein_sequence_len"]<=221]
train_df.reset_index(inplace=True)
sequences = [list(string) for string in train_df["protein_sequence"].values.tolist()]
sequences_train = pd.DataFrame(sequences)
sequences_train.head()

# Label encode <a class='anchor' id='labelencode'></a>[↑](#top)

We need to label encode data so we can use XGBoost Regressor

In [ ]:
from sklearn.preprocessing import LabelEncoder

sequences_train = sequences_train.apply(LabelEncoder().fit_transform)
sequences_train["tm"] = train_df["tm"]
sequences_train.head()

# XGBoost Regressor <a class='anchor' id='xgboost'></a>[↑](#top)

Let's make a 80/20 split and train a XGBoost regressor.

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost

X = sequences_train.loc[:, sequences_train.columns != "tm"]
y = sequences_train.loc[:, sequences_train.columns == "tm"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# create an xgboost regression model
model = xgboost.XGBRegressor(n_estimators=500, max_depth=15)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate: Spearman's rank correlation coefficient <a class='anchor' id='evaluate'></a>[↑](#top)

We need to evaluate with [Spearman's rank correlation coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)

In [ ]:
from scipy import stats

stats.spearmanr(y_test, y_pred)

# Predict <a class='anchor' id='predict'></a>[↑](#top)

Now that the model is trained let's convert the `test` dataframe into the required format and get predictions.

In [ ]:
from scipy.sparse import csr_matrix

test_df = test_df[test_df["protein_sequence_len"]<=221]
sequences = [list(string) for string in test_df["protein_sequence"].values.tolist()]
sequences_test = pd.DataFrame(sequences)
sequences_test = sequences_test.apply(LabelEncoder().fit_transform)
sequences_test.head()

# Submission <a class='anchor' id='submission'></a>[↑](#top)

Finally, let's submit our predictions.

In [ ]:
submission = pd.DataFrame()
submission["tm"] = model.predict(sequences_test)
submission["seq_id"] = test_df["seq_id"]
submission.to_csv("submission.csv", index=False)